<a href="https://colab.research.google.com/github/estermartins/Data-Preparation/blob/master/Exerc%C3%ADcio_Unbalanced.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Preparation

## Exercício CHURN - dados desbalanceados

Vamos fazer o download doa dados necessários para o exercício de hoje.

In [5]:
!wget http://www.vincentlemaire-labs.fr/kddcup2009/orange_small_train.data.zip  # melhora a taxa de transferência

--2019-06-05 00:21:56--  http://www.vincentlemaire-labs.fr/kddcup2009/orange_small_train.data.zip
Resolving www.vincentlemaire-labs.fr (www.vincentlemaire-labs.fr)... 81.169.145.72, 2a01:238:20a:202:1072::
Connecting to www.vincentlemaire-labs.fr (www.vincentlemaire-labs.fr)|81.169.145.72|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8424904 (8.0M) [application/zip]
Saving to: ‘orange_small_train.data.zip.1’

orange_small_train. 100%[===================>]   8.03M   635KB/s    in 13s     

2019-06-05 00:22:09 (625 KB/s) - ‘orange_small_train.data.zip.1’ saved [8424904/8424904]



In [6]:
!wget http://www.vincentlemaire-labs.fr/kddcup2009/orange_small_train_churn.labels

--2019-06-05 00:22:14--  http://www.vincentlemaire-labs.fr/kddcup2009/orange_small_train_churn.labels
Resolving www.vincentlemaire-labs.fr (www.vincentlemaire-labs.fr)... 81.169.145.72, 2a01:238:20a:202:1072::
Connecting to www.vincentlemaire-labs.fr (www.vincentlemaire-labs.fr)|81.169.145.72|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 196328 (192K)
Saving to: ‘orange_small_train_churn.labels.1’

orange_small_train_ 100%[===================>] 191.73K   629KB/s    in 0.3s    

2019-06-05 00:22:14 (629 KB/s) - ‘orange_small_train_churn.labels.1’ saved [196328/196328]



In [7]:
!unzip orange_small_train.data.zip

Archive:  orange_small_train.data.zip
replace orange_small_train.data? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [8]:
!ls

orange_small_train_churn.labels    orange_small_train.data.cksum  sample_data
orange_small_train_churn.labels.1  orange_small_train.data.zip
orange_small_train.data		   orange_small_train.data.zip.1


### Pré Processamento

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
# específicos
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix
from sklearn.impute import SimpleImputer

In [14]:
y = pd.read_csv("orange_small_train_churn.labels", header=None)
x = pd.read_csv("orange_small_train.data", sep = "\t") #sep = separador
print(y.shape)
print(x.shape)

(50000, 1)
(50000, 230)


In [15]:
y.head()

,0
0,-1
1,1
2,-1
3,-1
4,-1


In [16]:
x.head() 

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,Var11,Var12,Var13,Var14,Var15,Var16,Var17,Var18,Var19,Var20,Var21,Var22,Var23,Var24,Var25,Var26,Var27,Var28,Var29,Var30,Var31,Var32,Var33,Var34,Var35,Var36,Var37,Var38,Var39,Var40,...,Var191,Var192,Var193,Var194,Var195,Var196,Var197,Var198,Var199,Var200,Var201,Var202,Var203,Var204,Var205,Var206,Var207,Var208,Var209,Var210,Var211,Var212,Var213,Var214,Var215,Var216,Var217,Var218,Var219,Var220,Var221,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229,Var230
0,NaN,NaN,NaN,NaN,NaN,1526.0,7.0,NaN,NaN,NaN,NaN,NaN,184.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,464.0,580.0,NaN,14.0,128.0,NaN,NaN,166.56,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,3570.0,NaN,NaN,...,NaN,bZkvyxLkBI,RO12,NaN,taul,1K8T,lK27,ka_ns41,nQUveAzAF7,NaN,NaN,dXGu,9_Y1,FbIm,VpdQ,haYg,me75fM6ugJ,kIsH,NaN,uKAI,L84s,XfqtO3UdzaXh_,NaN,NaN,NaN,XTbPUYD,sH5Z,cJvF,FzaX,1YVfGrO,oslk,fXVEsaq,jySVZNlOJy,NaN,NaN,xb3V,RAYp,F2FyR07IdsN7I,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,525.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,168.0,210.0,NaN,2.0,24.0,NaN,NaN,353.52,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,4764966.0,NaN,NaN,...,NaN,CEat0G8rTN,RO12,NaN,taul,1K8T,2Ix5,qEdASpP,y2LIM01bE1,NaN,NaN,lg1t,9_Y1,k13i,sJzTlal,zm5i,me75fM6ugJ,kIsH,NaN,uKAI,L84s,NhsEn4L,NaN,NaN,NaN,kZJyVg2,NaN,NaN,FzaX,0AJo2f2,oslk,2Kb5FSF,LM8l689qOp,NaN,NaN,fKCe,RAYp,F2FyR07IdsN7I,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,5236.0,7.0,NaN,NaN,NaN,NaN,NaN,904.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1212.0,1515.0,NaN,26.0,816.0,NaN,NaN,220.08,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,5883894.0,NaN,NaN,...,NaN,eOQt0GoOh3,AERks4l,SEuy,taul,1K8T,ffXs,NldASpP,y4g9XoZ,vynJTq9,smXZ,4bTR,9_Y1,MGOA,VpdQ,haYg,DHn_WUyBhW_whjA88g9bvA64_,kIsH,NaN,uKAI,L84s,UbxQ8lZ,NaN,TTGHfSv,NaN,pMWAe2U,bHR7,UYBR,FzaX,JFM1BiF,Al6ZaUT,NKv4yOc,jySVZNlOJy,NaN,kG3k,Qu4f,02N6s8f,ib5G6X1eUxUn6,am7c,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,22.08,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,...,NaN,jg69tYsGvO,RO12,NaN,taul,1K8T,ssAy,_ybO0dd,4hMlgkf58mhwh,NaN,NaN,W8mQ,9_Y1,YULl,VpdQ,NaN,me75fM6ugJ,kIsH,NaN,uKAI,Mtgm,NhsEn4L,NaN,NaN,NaN,kq0dQfu,eKej,UYBR,FzaX,L91KIiz,oslk,CE7uk3u,LM8l689qOp,NaN,NaN,FSa2,RAYp,F2FyR07IdsN7I,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,1029.0,7.0,NaN,NaN,NaN,NaN,NaN,3216.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64.0,80.0,NaN,4.0,64.0,NaN,NaN,200.00,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,...,NaN,IXSgUHShse,RO12,SEuy,taul,1K8T,uNkU,EKR938I,ThrHXVS,0v21jmy,smXZ,xklU,9_Y1,RVjC,sJzTlal,6JmL,me75fM6ugJ,kIsH,NaN,uKAI,L84s,XfqtO3UdzaXh_,NaN,SJs3duv,NaN,11p4mKe,H3p7,UYBR,FzaX,OrnLfvc,oslk,1J2cvxe,LM8l689qOp,NaN,kG3k,FSa2,RAYp,F2FyR07IdsN7I,mj86,NaN


In [18]:
x.dtypes

Var1      float64
Var2      float64
Var3      float64
Var4      float64
Var5      float64
Var6      float64
Var7      float64
Var8      float64
Var9      float64
Var10     float64
Var11     float64
Var12     float64
Var13     float64
Var14     float64
Var15     float64
Var16     float64
Var17     float64
Var18     float64
Var19     float64
Var20     float64
Var21     float64
Var22     float64
Var23     float64
Var24     float64
Var25     float64
Var26     float64
Var27     float64
Var28     float64
Var29     float64
Var30     float64
           ...   
Var201     object
Var202     object
Var203     object
Var204     object
Var205     object
Var206     object
Var207     object
Var208     object
Var209    float64
Var210     object
Var211     object
Var212     object
Var213     object
Var214     object
Var215     object
Var216     object
Var217     object
Var218     object
Var219     object
Var220     object
Var221     object
Var222     object
Var223     object
Var224     object
Var225    

In [20]:
x = x .iloc[:, :50] # diferença de iloc por loc
x.shape

(50000, 50)

In [23]:
# Como resolver nas de variavel numeirca = imputar media
imputer = SimpleImputer(missing_values=np.nan, strategy="mean")
x =imputer.fit_transform(x)
pd.DataFrame(x).describe()      # se tiver algum NA a contagem vai ser menor que 50.000

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41
count,50000.000000,50000.000000,50000.000000,50000.000000,5.000000e+04,50000.000000,50000.000000,50000.000000,5.000000e+04,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,5.000000e+04,50000.000000,50000.000000,5.000000e+04,5.000000e+04,5.000000e+04,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000
mean,11.487179,0.004029,425.298387,0.125396,2.387933e+05,1326.437116,6.809496,48.145299,3.926057e+05,8.625806,16.071685,1249.688401,0.741935,120.239275,11.393287,7.215959,0.245092,234.518225,290.245382,7.535306,4.507926,96.827010,0.070612,0.028245,224.507669,0.022792,7.435897,1.270050e+05,1.173247,0.716810,1.595539e+05,6.485221e+05,2.579107e+06,13.962933,26.653846,4.157937,0.166833,7256.127907,16.796132,2.188034,0.091861,35.690883
std,4.820354,0.022352,672.206258,0.226593,1.110451e+05,2532.849475,5.965363,18.326822,1.599995e+05,0.451721,6.774604,2635.596185,0.584701,12.479146,8.792738,6.114008,0.316489,533.373831,668.270306,8.524922,9.182934,203.299616,0.091170,0.042618,93.452978,0.025154,1.048196,6.449174e+04,0.689409,2.841975,5.160914e+04,2.455565e+05,2.855321e+06,12.820589,7.322959,3.239461,1.545575,728.717325,11.152436,1.206654,0.138676,15.041819
min,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,8.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-66.880000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,11.487179,0.004029,425.298387,0.125396,2.387933e+05,581.000000,0.000000,48.145299,3.926057e+05,8.625806,16.071685,4.000000,0.741935,120.239275,11.393287,7.215959,0.245092,120.000000,150.000000,7.535306,0.000000,16.000000,0.070612,0.028245,166.560000,0.022792,7.435897,1.270050e+05,1.173247,0.000000,1.595539e+05,6.485221e+05,1.762500e+04,13.962933,26.653846,4.157937,0.000000,7256.127907,16.796132,2.188034,0.091861,35.690883
50%,11.487179,0.004029,425.298387,0.125396,2.387933e+05,945.000000,7.000000,48.145299,3.926057e+05,8.625806,16.071685,376.000000,0.741935,120.239275,11.393287,7.215959,0.245092,156.000000,190.000000,7.535306,2.000000,56.000000,0.070612,0.028245,220.080000,0.022792,7.435897,1.270050e+05,1.173247,0.000000,1.595539e+05,6.485221e+05,2.073606e+06,13.962933,26.653846,4.157937,0.000000,7256.127907,16.796132,2.188034,0.091861,35.690883
75%,11.487179,0.004029,425.298387,0.125396,2.387933e+05,1326.437116,7.000000,48.145299,3.926057e+05,8.625806,16.071685,1264.000000,0.741935,120.239275,11.393287,7.215959,0.245092,234.518225,290.245382,7.535306,4.507926,104.000000,0.070612,0.028245,256.960000,0.022792,7.435897,1.270050e+05,1.173247,0.000000,1.595539e+05,6.485221e+05,4.241100e+06,13.962933,26.653846,4.157937,0.000000,7256.127907,16.796132,2.188034,0.091861,35.690883
max,680.000000,5.000000,130668.000000,27.000000,6.048550e+06,131761.000000,140.000000,2300.000000,1.232559e+07,40.000000,1184.000000,197872.000000,48.000000,434.920000,1220.000000,948.000000,27.000000,36272.000000,45340.000000,1555.000000,494.000000,13168.000000,9.000000,4.000000,5158.560000,2.000000,95.000000,1.088640e+07,56.000000,110.000000,2.419600e+06,1.163502e+07,1.884690e+07,2648.000000,728.000000,625.000000,135.000000,60553.400000,1668.000000,168.000000,18.000000,1660.000000


### Elaboração dos modelos

In [0]:
x_train, x_test, y_train, y_test = train_test_split(x,y , test_size=.3, random_state=3)

In [27]:
# Elaborar um modelo de reg Log
# Instanciar uma classe do modelo, fit, predict e testar o erro
model = LogisticRegressionCV(cv=10)
model.fit(x_train, y_train)
yhat = model.predict(x_test)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of

In [28]:
# Avaliar os resultados
# acuracia
# metrica da acuracia = testa a quantidade de testes dentro da quantidade de acertos
# modelo com 92% de acuracia
accuracy_score(y_test, yhat)

0.9274

In [29]:
# area debaixo da curva
# 0,5 ajuste pessimo > 1 ajuste otimo
roc_auc_score(y_test, yhat)
# resultado= modelo pessimo = 0,5

0.5

In [30]:
confusion_matrix(y_test, yhat)
# 13000 casos que eram negativos, e ele acertou a previsao # 1089 eram positivos e nao acertou a previsao

array([[13911,     0],
       [ 1089,     0]])

#### O MODELO APRESENTA QUEM NAO QUER CANCELAR O PACOTE DE SERVIÇOS, E NÃO É ESSA A PREVISÃO DESEJADA. 

In [31]:
# Observando com mais cuidade a variável resposta
y[0].value_counts() / y[0].value_counts().sum()

-1    0.92656
 1    0.07344
Name: 0, dtype: float64

In [32]:
# Balancear a variável resposta com SMOTE = técnica de oversampling
# replica dados # fabrica dados com "sujeiras"
sm = SMOTE(random_state=3)
### IMP não faz sentido balancear dados de teste, pois dados de teste so vai ser feito um teste. Apenas dados de TREINO(train)
x_res, y_res = sm.fit_resample(x_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [33]:
pd.DataFrame(y_res)[0].value_counts()
# replicou

-1    32417
 1    32417
Name: 0, dtype: int64

In [0]:
modelsm = LogisticRegressionCV(cv=10)
modelsm.fit(x_res, y_res)
yhatsm = modelsm.predict(x_test)

In [35]:
# Avaliando o ajudte dos resultados
accuracy_score(y_test, yhatsm)

0.4391333333333333

In [36]:
roc_auc_score(y_test, yhatsm)

0.547379513962532

In [37]:
confusion_matrix(y_test, yhatsm) # errando mais casos negativos, que ele deduz ser positivos # mas está acertando alguns casos positivos
# melhorou a previsão para quem deseja cancelar o serviço

array([[5853, 8058],
       [ 355,  734]])

Pegar o banco de dados da orange e indicar qual os tres indicadores chaves 

------- diferença ter melhor pre processamento